In [13]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash.dependencies import Input, Output, State


import plotly.graph_objects as go
import plotly.figure_factory as ff

import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
#データ取得
df = pd.read_csv("./data/Hotel Reservations.csv").sample(frac=0.1, random_state=42)
df.head(3)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
4968,INN04969,2,1,1,0,Meal Plan 1,0,Room_Type 1,3,2017,8,23,Online,0,0,0,90.00,3,Not_Canceled
34540,INN34541,2,0,1,2,Meal Plan 1,0,Room_Type 4,9,2018,2,12,Offline,0,0,0,48.67,0,Not_Canceled
36108,INN36109,2,0,2,2,Meal Plan 1,0,Room_Type 1,24,2018,12,25,Online,0,0,0,95.20,1,Not_Canceled


In [15]:
#グラフ作成用に特徴量を整理
hist_scatter_cols = ["lead_time", "arrival_date","no_of_previous_bookings_not_canceled","avg_price_per_room",  ]
bar_cols = ["no_of_adults", "no_of_children", "no_of_weekend_nights", "type_of_meal_plan", \
           "required_car_parking_space", "room_type_reserved", "arrival_year", "arrival_month", \
           "market_segment_type", "repeated_guest", "no_of_previous_cancellations", "no_of_special_requests"]

In [16]:
#カラーコード
back_color = "#f6f5f5"
plot_bg_color = "#d8d2cb"
cancel_color = "#ee6f57"
not_cancel_color = "#145374"

### 円グラフ

In [17]:
#円グラフ作成
#割合を計算
pie = df.groupby("booking_status").count()["Booking_ID"]/len(df)

#円グラフのfigure
fig_pie = go.Figure(
    data = [go.Pie(labels=list(pie.index),#label
                   values=pie.values,#値
                   hole=0.2,#穴の大きさ
                   marker=dict(colors=[cancel_color, not_cancel_color]))])#色指定

#円グラフのレイアウト設定
fig_pie.update_layout(
    legend=dict(x=1, y=0.5,
                xanchor="left",
                yanchor="top",
                orientation="h"),
    width=500,
    height=300,
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="rgba(0,0,0,0)",
)

## appのインスタンス化

In [18]:
#インスタンス化、テーマ設定
app = dash.Dash(external_stylesheets=[dbc.themes.FLATLY])

# サーバー
server = app.server

# ダッシュボード

In [19]:
#LeftAreaのレイアウト
LeftArea = html.Div(
    [
        dbc.Row(#見出し
            [
                html.Div([
                    html.H5('Target Ratio', className='font-weight-bold'),
                    ])
            ],
            style={"height": "4vh",
                  "width":800}, className='bg-black text-white'
            ),
        dbc.Row(#円グラフ
            [
                html.Div([
                    dcc.Graph(figure=fig_pie)
                    ])
            ],
            style={"height": "40vh",
                  "width":500,
                  "margin-left" :100,
                  "margin-top" : 10}
            ),
        dbc.Row(#見出し
            [
                html.H5('Scatter Plot', className='font-weight-bold')
                ],
          style={"height": "4vh",
                  "width":800}, className='bg-black text-white'
            ),
        dbc.Row([#軸名の記載
                html.Div([
                    html.H6('X Axis :', className='font-weight-bold'),
                    ],style={"width" : 100,
                            "margin-top" : 5}),
            #ドロップダウン x軸
            dcc.Dropdown(id='scatter_x', multi=False, value='avg_price_per_room',
                                 options=[{'label': x, 'value': x}
                                          for x in hist_scatter_cols],
                                 style={'width': 400,
                                       "margin-top":1}
                                 )
                
                ]
           
            ),
        dbc.Row([#軸名の記載
            html.Div([
                    html.H6('Y Axis :', className='font-weight-bold'),
                    ],style={"width" : 100,
                            "margin-top" : 5}),
            #ドロップダウン y軸
                 dcc.Dropdown(id='scatter_y', multi=False, value='lead_time',
                                 options=[{'label': x, 'value': x}
                                          for x in hist_scatter_cols],
                                 style={'width': 400,
                                       "margin-top":1}
                                 ),
            #applyボタン
             html.Button(id='scatter-button', n_clicks=0, children='apply',
                                 style={'width': 100,
                                       "margin-left":30,
                                       "margin-bottom":5},
                                className='bg-dark text-white')
                
                ]
           
            ),
        dbc.Row(#散布図
            [
                html.Div([
                    dcc.Graph(id="scatter_chart",
                        figure={})
                    ])
            ],
            style={"height": "39.83vh",
                   "width" : 700,
                  "margin-left" : 0,
                  "margin-top" : 0}
            ),
        ]
    )
#散布図
@app.callback(
    Output('scatter_chart', 'figure'),
    Input('scatter-button', 'n_clicks'),
    State('scatter_x', 'value'),
    State('scatter_y', 'value')
)
def update_graph(n_clicks, scatter_x, scatter_y):
    fig_scatter = go.Figure()
    fig_scatter.add_trace(go.Scatter(name='Not_Canceled',
                                     x=df.query('booking_status=="Not_Canceled"')[scatter_x], 
                                     y=df.query('booking_status=="Not_Canceled"')[scatter_y],
                                     mode='markers',
                                     marker={"color" : not_cancel_color})
                      )
    fig_scatter.add_trace(go.Scatter(name='Canceled',
                                     x=df.query('booking_status=="Canceled"')[scatter_x], 
                                     y=df.query('booking_status=="Canceled"')[scatter_y],
                                     mode='markers', marker={"opacity":0.5,
                                                             "color":cancel_color})
                      )
    fig_scatter.update_layout(
        height=300,
        margin=dict(
            l=10,  # 左側の余白
            r=10,  # 右側の余白
            b=0,  # 下側の余白
            t=10   # 上側の余白
        ),
        xaxis=dict(title=scatter_x),
        yaxis=dict(title=scatter_y),
        plot_bgcolor=plot_bg_color,  # グラフエリアの背景色
        paper_bgcolor=back_color  # グラフ全体の背景色
    )
    return fig_scatter


In [20]:
#RightAreaのレイアウト
RightArea = html.Div(
    [
        dbc.Row(#見出し
            [
                html.H5('Bar Plot', className='font-weight-bold')
                ],
            style={"height": "4vh",
                   "width" : 738}, className='bg-black text-white'
            
            ),
        dbc.Row(#ドロップダウン
            [dcc.Dropdown(id='bar_chart_drop', multi=False, value='no_of_adults',
                                 options=[{'label': x, 'value': x}
                                          for x in bar_cols],
                                 style={'width': 320,
                                       "margin-top":1,
                                       "margin-left":10}
                                 )
            ]
            ),
        dbc.Row(#棒グラフ
            [
                html.Div([
                    dcc.Graph(id="bar_chart",
                              figure={})
                    ])
            ],
            style={"height": "36.2vh",
                   "width" : 700,
                  "margin-left" : 0,
                  "margin-top" : 2,
                  "margin-bottom":0}
            ),
        dbc.Row(#見出し
            [
                html.H5('Distogram', className='font-weight-bold')
                ],
            style={"height": "4vh",
                   "width" : 738,
                  "margin-top":0}, className='bg-black text-white'
            ),
        dbc.Row(#ドロップダウン
            [dcc.Dropdown(id='dist_chart_drop', multi=False, value='lead_time',
                                 options=[{'label': x, 'value': x}
                                          for x in hist_scatter_cols],
                                 style={'width': 320,
                                       "margin-top":1,
                                       "margin-left":10}
                                 )
            ]
            ),
        dbc.Row(#Distgram
            [
                html.Div([
                    dcc.Graph(id="dist_chart",
                              figure={})
                    ])
            ],
            style={"height": "39.83vh",
                   "width" : 700,
                  "margin-left" : 0,
                  "margin-top" : 2,
                  "margin-bottom":0}
            )
        
        ]
    )

#棒グラフ
@app.callback(
    Output('bar_chart', 'figure'),
    Input('bar_chart_drop', 'value')
)
def update_graph(bar_col):
    fig_bar = go.Figure()
    fig_bar.add_trace(go.Bar(name='Not_Canceled',
                        x=df.query('booking_status=="Not_Canceled"')[bar_col].value_counts().index,
                        y=df.query('booking_status=="Not_Canceled"')[bar_col].value_counts(),
                        marker={"color" : not_cancel_color}
                        ))
    fig_bar.add_trace(go.Bar(name='Canceled',
                        x=df.query('booking_status=="Canceled"')[bar_col].value_counts().index,
                        y=df.query('booking_status=="Canceled"')[bar_col].value_counts(),
                        marker={"color" : cancel_color},
                        ))

    fig_bar.update_layout(
        height=280,
        margin=dict(
            l=10,  # 左側の余白
            r=10,  # 右側の余白
            b=10,  # 下側の余白
            t=10   # 上側の余白
        ),
        plot_bgcolor=plot_bg_color,  # グラフエリアの背景色
        paper_bgcolor=back_color  # グラフ全体の背景色
    )
    return fig_bar

#ヒストグラム
@app.callback(Output('dist_chart', 'figure'),Input('dist_chart_drop', 'value')
)
def update_graph(dist_col):
    num0 = df[df['booking_status'] == "Canceled"][dist_col].values.tolist()
    num1 = df[df['booking_status'] == "Not_Canceled"][dist_col].values.tolist()
    fig_dist = ff.create_distplot(hist_data=[num0, num1],
                                  group_labels=["Canceled", "Not_Canceled"],
                                  show_hist=False,
                                  colors=[cancel_color, not_cancel_color])
    fig_dist.update_layout(height=340,margin=dict(
            l=10,  # 左側の余白
            r=10,  # 右側の余白
            b=10,  # 下側の余白
            t=10   # 上側の余白
    ),
        plot_bgcolor=plot_bg_color,  # グラフエリアの背景色
        paper_bgcolor=back_color  # グラフ全体の背景色
                          )
    return fig_dist

In [21]:
#全体のレイアウト
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(LeftArea, width=6, className='bg-white'),
                dbc.Col(RightArea, width=6, className='bg-white')
                ],
            ),
        ],
    fluid=True
    )

In [22]:
#サーバー実行
if __name__ == "__main__":
    app.run_server(port = 8055)

http://127.0.0.1:8055/